# Importing and preprocessing of data

In [1]:
from google.colab import drive

drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
TXTS_PATH = '/content/drive/My Drive/Becode/Kpmg/selection_fr'

In [3]:
import os

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', 255)

In [5]:
KPMG_PATH = '/content/drive/My Drive/Becode/Kpmg'
RESPONSES_PATH = '/content/drive/My Drive/Becode/Kpmg/responses_2018_now.json'

In [6]:
txt_filenames = [f for f in os.listdir(TXTS_PATH) if f.endswith(".txt")]

In [7]:
#reading json file to retrieve tags
responses = pd.read_json(RESPONSES_PATH)

In [8]:
#filtering only relevant columns
columns = ['jcId', 'jcFr', 'titleFr', 'themesFr', 'scopeFr', 'noScopeFr', 'documentLink']
responses2 = responses.copy(deep=True).loc[:,columns]
#getting txt_name from original pdf name
responses2["txt_name"] = responses2["documentLink"]
responses2["txt_name"] = responses2["txt_name"].str.replace("/","-")
responses2["txt_name"] = responses2["txt_name"].str.replace(".pdf","_FR.txt")
#filtering json only for selected files and columns
columns += ['txt_name']
responses2 = responses2.loc[responses2.txt_name.isin(txt_filenames), columns]
responses2.head(2)

,jcId,jcFr,titleFr,themesFr,scopeFr,noScopeFr,documentLink,txt_name
34,1110000,"COMMISSION PARITAIRE DES CONSTRUCTIONS METALLIQUE, MECANIQUE ET ELECTRIQUE","modification du régime de pension sectoriel social, du règlement de pension et du règlement de solidarité",[PENSIONS COMPÉMENTAIRES ET ASSURANCES GROUPES],None,"[les employeurs et ouvriers des entreprises exemptées du paiement d'une cotisation pour le régime de pension sectorielsur la base d'un accord d'entreprise relatif à l'instauration ou à l'élargissement d'un régime de pension complémentaire, employeurs ...",111/111-2018-013525.pdf,111-111-2018-013525_FR.txt
35,1110000,"COMMISSION PARITAIRE DES CONSTRUCTIONS METALLIQUE, MECANIQUE ET ELECTRIQUE",allocation spéciale compensatoire,[PRIME SYNDICALE],None,None,111/111-2018-012196.pdf,111-111-2018-012196_FR.txt


In [9]:
#generating all possible FR themes
themes_fr = []
for r in  responses2.themesFr:
    if r is not None:
        for t in r:
            if t not in themes_fr:
                themes_fr += [t]
len(themes_fr)


54

In [10]:
#code to retrieve the entire body
#%%
for filename in responses2.txt_name.to_list():
    # filename = responses2.txt_name.to_list()[10] #test
    file_path = os.path.join(TXTS_PATH, filename)
    with open(file_path, 'r', encoding="utf8") as f:
        #to retrieve entire body not necessary
        responses2.loc[responses2.txt_name == filename, "doc_bodies"] = f.read()
        f.close()

In [11]:
t = themes_fr[0]
mask = responses2.dropna(axis=0, subset=['themesFr']).themesFr.map(lambda x: t in x)

In [12]:
len(responses2)

578

In [13]:
 #len(responses2.dropna(axis=0, subset=['themesFr'])[map])

# Regex extraction
[RegEx Match Object](https://www.w3schools.com/python/gloss_python_regex_match.asp)

In [14]:
"""This script regex-extracts information from the document bodies to new DataFrame columns"""
#redundant libraries to be checked
import datetime
from pickle import load
import re

import numpy as np
import pandas as pd

In [15]:
def split_func(text):
  texts_lst = []
  sents = text.split("\n\n")
  for sent in sents:
    texts_lst.append(sent)
  return texts_lst

In [17]:
def extract_pattern(lst, pattern:str):
  output_lst = []
  for item in lst: 
    if re.search(pattern, item, flags=re.IGNORECASE):
      output_lst.append(item)
    else:
      pass
  return '\n'.join(output_lst)

In [18]:
def extract_scope(lst):
  output_lst = []
  for item in lst: 
    #pattern1 = r".*\b(convention collective de travail s'applique | Champ d'application)\b.*"
    #pattern2 = r".*\b(La présente convention collective de travail\s*\S* [a-z]'appli\w*)\b.*"
    #pattern3 = r".*\b(convention collective de travail|CCT)\s*\S*[a-z]('appli\w*)\b.*"
    #pattern4 = r".*\b(convention collective de travail|CCT)\s*\S*.*('?appli\w*)\b.*"
    #pattern = r".*\b(convention collective de.*travail|CCT).*('*appli\w*)\b.*"
    pattern = r".*((convention collective de )(travail|CCT))*.*"
    if re.search(pattern, item, flags=re.IGNORECASE):
      output_lst.append(item)
    else:
      pass
  return '\n'.join(output_lst)

In [68]:
def extract_deposit(lst):
  output_lst = []
  for item in lst: 
    pattern =r".*(épôt: )([0-9]{2})/([0-9]{2})/([0-9]{4}).*"
    match = re.search(pattern, item, flags=re.IGNORECASE)
    if match:
      output_str = str(match[2])+"/"+str(match[3])+"/"+str(match[])
      output_lst.append(output_str)
    else:
      pass
  return '\n'.join(output_lst)

In [ ]:
#Pendant la durée de cette CCT tous les jours de chômage temporaire ne seront pas pris en compte dans le calcul de la période de maximum soixante jours par année de article 2 de la CCT du 17 septembre 2019 conclue au sein de la Commission Paritaire de l’industrie chimique relative à la sécurité d'existence (numéro d'enregistrement 154419).



In [20]:
def add_tags(df, body_column):
  df['texts_lst'] = df[body_column].apply(split_func)
  #pattern1 = r".*\b(convention collective de travail s'applique | Champ d'application)\b.*
  #pattern2 = r".*\b(La présente convention collective de travail\s*\S* [a-z]'appli\w*)\b.*"
  #pattern3 = r".*\b(convention collective de travail|CCT)\s*\S*[a-z]('appli\w*)\b.*"
  #pattern4 = r".*\b(convention collective de travail|CCT)\s*\S*.*('?appli\w*)\b.*"
  #pattern = r".*\b(convention collective de.*travail|CCT).*('*appli\w*)\b.*"
  df['scopeFr'] = df['texts_lst'].apply(extract_scope)
  df['depotFr'] = df['texts_lst'].apply(extract_deposit)
  df.drop(columns=['texts_lst'], inplace = True)
  return df

In [64]:
df = responses2.head(5)
df['texts_lst'] = df["doc_bodies"].apply(split_func)
df['texts_lst']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


34    [Neerlegging-Dépôt: 18/12/2018, Regist.-Enregistr.: 07/01/2019, N°: 149887/C0/111 , COMMISSION PARITAIRE DES CONSTRUCTIONS |, METALLIQUE, MECANIQUE ET ELECTRIQUE (PC |, Mt) , | , | , Ì       , CONVENTION COLLECTIVE DE TRAVAIL DU 17, DECEMBRE 2018 MODI...
35    [COMMISSION PARITAIRE 111 DES CONSTRUCTIONS, MÉTALLIQUE, MÉCANIQUE ET ÉLECTRIQUE , Convention collective de travail du 15 octobre 2018 , ALLOCATION SPECIALE COMPENSATOIRE , Chapitre ler. - Champ d'application , Article 1er. - La présente convention co...
36    [Neerlegging-Dépôt: 17/10/2018, Regist.-Enregistr.: 05/11/2018, N°: 148625/CO/111 , COMMISION PARITAIRE DE LA CONSTRUCTION, METALLIQUE, MECANIQUE ET ELECTRIQUE SECTION MONTAGE (CP 111.3) , Convention collective de travail du 15 octobre 2018 , MODIFICA...
37    [Neerlegging-Dépôt: 05/09/2018, Regist.-Enregistr.: 17/10/2018, N°: 148350/C0/111 , Commission paritaire des constructions métallique, mécanique et électrique , Convention collective de travail du 3 septembre 20

In [69]:
df['depotFr'] = df['texts_lst'].apply(extract_deposit)
df['depotFr']
#df = add_tags(df, "doc_bodies")
#df.loc[:,['txt_name','depotFr']]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


34    18/12/2018
35              
36    17/10/2018
37    05/09/2018
38    03/07/2018
Name: depotFr, dtype: object

In [25]:
df = add_tags(df, 'doc_bodies')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,jcId,jcFr,titleFr,themesFr,scopeFr,noScopeFr,documentLink,txt_name,doc_bodies,depotFr
34,1110000,"COMMISSION PARITAIRE DES CONSTRUCTIONS METALLIQUE, MECANIQUE ET ELECTRIQUE","modification du régime de pension sectoriel social, du règlement de pension et du règlement de solidarité",[PENSIONS COMPÉMENTAIRES ET ASSURANCES GROUPES],"Neerlegging-Dépôt: 18/12/2018\nRegist.-Enregistr.: 07/01/2019\nN°: 149887/C0/111 \nCOMMISSION PARITAIRE DES CONSTRUCTIONS |\nMETALLIQUE, MECANIQUE ET ELECTRIQUE (PC |\nMt) \n| \n| \nÌ \nCONVENTION COLLECTIVE DE TRAVAIL DU 17\nDECEMBRE 2018 MODIF...","[les employeurs et ouvriers des entreprises exemptées du paiement d'une cotisation pour le régime de pension sectorielsur la base d'un accord d'entreprise relatif à l'instauration ou à l'élargissement d'un régime de pension complémentaire, employeurs ...",111/111-2018-013525.pdf,111-111-2018-013525_FR.txt,"Neerlegging-Dépôt: 18/12/2018\n\nRegist.-Enregistr.: 07/01/2019\n\nN°: 149887/C0/111 \n\nCOMMISSION PARITAIRE DES CONSTRUCTIONS |\n\nMETALLIQUE, MECANIQUE ET ELECTRIQUE (PC |\n\nMt) \n\n| \n\n| \n\nÌ \n\nCONVENTION COLLECTIVE DE TRAVAIL DU 17\n\...",Neerlegging-Dépôt: 18/12/2018
35,1110000,"COMMISSION PARITAIRE DES CONSTRUCTIONS METALLIQUE, MECANIQUE ET ELECTRIQUE",allocation spéciale compensatoire,[PRIME SYNDICALE],"COMMISSION PARITAIRE 111 DES CONSTRUCTIONS\nMÉTALLIQUE, MÉCANIQUE ET ÉLECTRIQUE \nConvention collective de travail du 15 octobre 2018 \nALLOCATION SPECIALE COMPENSATOIRE \nChapitre ler. - Champ d'application \nArticle 1er. - La présente convention col...",None,111/111-2018-012196.pdf,111-111-2018-012196_FR.txt,"COMMISSION PARITAIRE 111 DES CONSTRUCTIONS\n\nMÉTALLIQUE, MÉCANIQUE ET ÉLECTRIQUE \n\nConvention collective de travail du 15 octobre 2018 \n\nALLOCATION SPECIALE COMPENSATOIRE \n\nChapitre ler. - Champ d'application \n\nArticle 1er. - La présente conv...",
36,1110000,"COMMISSION PARITAIRE DES CONSTRUCTIONS METALLIQUE, MECANIQUE ET ELECTRIQUE",modification et coordination de la cct relative aux primes,"[TRAVAIL EN ÉQUIPE ET DE NUIT, PÉCULE DE VACANCES, PRIME PROPRE AU SECTEUR OU À L'ENTREPRISE, REMBOURSEMENT DE FRAIS (HORS FRAIS DE DÉPLACEMENT), DÉTACHEMENT]","Neerlegging-Dépôt: 17/10/2018\nRegist.-Enregistr.: 05/11/2018\nN°: 148625/CO/111 \nCOMMISION PARITAIRE DE LA CONSTRUCTION\nMETALLIQUE, MECANIQUE ET ELECTRIQUE SECTION MONTAGE (CP 111.3) \nConvention collective de travail du 15 octobre 2018 \nMODIFICAT...",[secteur des entreprises de fabrications métalliques],111/111-2018-012197.pdf,111-111-2018-012197_FR.txt,"Neerlegging-Dépôt: 17/10/2018\n\nRegist.-Enregistr.: 05/11/2018\n\nN°: 148625/CO/111 \n\nCOMMISION PARITAIRE DE LA CONSTRUCTION\n\nMETALLIQUE, MECANIQUE ET ELECTRIQUE SECTION MONTAGE (CP 111.3) \n\nConvention collective de travail du 15 octobre 2018 \...",Neerlegging-Dépôt: 17/10/2018
37,1110000,"COMMISSION PARITAIRE DES CONSTRUCTIONS METALLIQUE, MECANIQUE ET ELECTRIQUE",addendum à la cct du 16/10/2017 concernant la prime de fin d'année en province d'Anvers,[PRIME DE FIN D'ANNÉE],"Neerlegging-Dépôt: 05/09/2018\nRegist.-Enregistr.: 17/10/2018\nN°: 148350/C0/111 \nCommission paritaire des constructions métallique, mécanique et électrique \nConvention collective de travail du 3 septembre 2018 \nAddendum à la CCT du 16 octobre 2017...",[les entreprises de montage de ponts et charpentes métalliques],111/111-2018-011427.pdf,111-111-2018-011427_FR.txt,"Neerlegging-Dépôt: 05/09/2018\n\nRegist.-Enregistr.: 17/10/2018\n\nN°: 148350/C0/111 \n\nCommission paritaire des constructions métallique, mécanique et électrique \n\nConvention collective de travail du 3 septembre 2018 \n\nAddendum à la CCT du 16 oc...",Neerlegging-Dépôt: 05/09/2018
38,1110000,"COMMISSION PARITAIRE DES CONSTRUCTIONS METALLIQUE, MECANIQUE ET ELECTRIQUE",salaire minimum garanti,"[SALAIRES, DÉTACHEMENT]","Neerlegging-Dépôt: 03/07/2018\nRegist.-Enregistr.: 14/08/2018\nN°: 147257/CO/111 \nCOMMISSION PARITAIRE DES CONSTRUCTIONS\nMETALLIQUE, MECANIQUE ET ELECTRIQUE \nCon

In [ ]:
def get_month_digit(month):
            dictionary = {
                'janvier': 1,
                'février': 2,
                'mars': 3,
                'avril': 4,
                'mai': 5,
                'juin': 6,
                'juillet': 7,
                'août': 8,
                'septembre': 9,
                'octobre': 10,
                'novembre': 11,
                'décembre': 12
            }            
            return dictionary[month]

In [59]:

def add_effective_date(df, body_column):
    """A function that searches the document bodies for fromDate and endDates
    and extracts datetime objects into new columns"""
    
    def get_dates(df_row):
        #start date matches ~46 out of 48 test cases
        #group 6 = month
        #group 7 = year
        pattern = r"((cette|la présente) (convention collective de travail|CCT)|elle).+(entre en vigueur|à partir|produit ses effets|s'étend|sort ses effets|prend cours|\bdéterminée|allant).{1,19}(le\b|du\b|de\b|au\b).{1,5}(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre) ([0-9]{4})"
        pattern = r"(cette|la présente) (convention collective de travail|CCT|elle).+(entre en vigueur|à partir|produit ses effets|s'étend|sort ses effets|prend cours|\bdéterminée|allant).{1,19}(le\b|du\b|de\b|au\b).{1,5}(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre) ([0-9]{4})"
        #pattern = r"[cette|la présente] [convention collective de travail|CCT]|[elle].+[entre en vigueur|à partir|produit ses effets|s'étend|sort ses effets|prend cours|\bdéterminée|allant].{1,19}[le\b|du\b|de\b|au\b].{1,5}(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre) ([0-9]{4})"
        
        # finds the first instance of the regex in the body
        match = re.search(pattern, df_row[body_column], flags=re.IGNORECASE)
        if match:
            try:
                #df_row['fromDate'] = datetime.date(int(match[7]), get_month_digit(match[6]), 1)
                df_row['fromDate'] = datetime.date(int(match[6]), get_month_digit(match[5]), 1)
                #df_row['fromDate'] = datetime.date(int(match[1]), get_month_digit(match[0]), 1)
                #testing purpose fm
                print(match[0])
                print(match[1])
                print(match[2])
                print(match[3])
                print(match[4])
                print(match[5])
                print(match[6])
                #print(f'{type(match)}')
                #for i in range(10):
                #  print(match[i])
            except:
                pass
        else:
            df_row['fromDate'] = np.nan
        
        #end date matches 47 of 48
        #group 6 = month
        #group 7 = year
        pattern = r"((cette|la présente) (convention collective de travail|CCT)|elle).+([0-9]{4}|cesse de produire ses effets|cesse d'être en vigueur|cesse ses effets|prend fin|expire|conclue jusq|prend.{1,25}fin).+(le\b|au\b).{1,5}(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre) ([0-9]{4})"
        pattern = r"(cette|la présente) (convention collective de travail|CCT|elle).+([0-9]{4}|cesse de produire ses effets|cesse d'être en vigueur|cesse ses effets|prend fin|expire|conclue jusq|prend.{1,25}fin).+(le\b|au\b).{1,5}(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre) ([0-9]{4})"
        #pattern = r"[cette|la présente] [convention collective de travail|CCT)|elle].+[[0-9]{4}|cesse de produire ses effets|cesse d'être en vigueur|cesse ses effets|prend fin|expire|conclue jusq|prend.{1,25}fin].+[le\b|au\b].{1,5}(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre) ([0-9]{4})"
        #pattern = r"[cette|la présente] [convention collective de travail|CCT)|elle].+[[0-9]{4}|cesse de produire ses effets|cesse d'être en vigueur|cesse ses effets|prend fin|expire|conclue jusq|prend.{1,25}fin]"
        #pattern += r".+[le\b|au\b].{1,5}(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre) ([0-9]{4})"
        
        match = re.search(pattern, df_row[body_column], flags=re.IGNORECASE)
        if match:
            try:
                #df_row['endDate'] = datetime.date(int(match[7]), get_month_digit(match[6]), 28)
                df_row['endDate'] = datetime.date(int(match[6]), get_month_digit(match[5]), 28)
                #df_row['endDate'] = datetime.date(int(match[1]), get_month_digit(match[0]), 28)
                #testing purpose fm   
                #print(f'{type(match)}') 
                print(match[0])
                print(match[1])
                print(match[2])
                print(match[3])
                print(match[4])
                print(match[5])
                print(match[6])     
                #for i in range(10):
                #  print(match[i])
            except:
                pass
        else:
            df_row['endDate'] = np.nan
        
        #there might have been an erroneous match of endDate
        #which should be overwritten by None when durée idéterminée is detected
        pattern = r"((cette|la présente) (convention collective de travail|CCT)|elle).+(durée indéterminée|([0-9]{4}.+à l'exception))"
        match = re.search(pattern, df_row[body_column], flags=re.IGNORECASE)
        if match:
            df_row['endDate'] = np.nan
        
        return df_row
    
    df = df.apply(get_dates, axis=1)

    return df

In [71]:
#extracting reference
#Pendant la durée de cette CCT tous les jours de chômage temporaire ne seront pas pris en compte dans le calcul de la période de maximum soixante jours par année de article 2 de la CCT du 17 septembre 2019 conclue au sein de la Commission Paritaire de l’industrie chimique relative à la sécurité d'existence (numéro d'enregistrement 154419).

def extract_reference(lst):
  output_lst = []
  for item in lst: 
    pattern =r".*(article )([0-9]{2}).*([0-9]{2}) (janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre) ([0-9]{4}).*(numéro d'enregistrement)([0-9]{6}).*"
    match = re.search(pattern, item, flags=re.IGNORECASE)
    if match:
      output_str = "article "+str(match[2])+","+str(match[3])+"/"+get_month_digit(match[4])+"/"+str(match[5])+", n. "+match[7]
      output_lst.append(output_str)
    else:
      pass
  return '\n'.join(output_lst)


In [73]:
df['reference'] = df['texts_lst'].apply(extract_reference)
df['reference']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


34    
35    
36    
37    
38    
Name: reference, dtype: object

In [27]:
def extract(df):
	'''overall regex extraction functioin'''

	df = add_scope(df)

	df = add_effective_date(df)

	return df

In [60]:
df =  add_effective_date(df, "doc_bodies")
#df.loc[:,['txt_name', 'texts_lst', 'depotFr', 'fromDate', 'endDate']]


La présente convention collective de travail, remplace à partir du 1” janvier 2018
La présente
convention collective de travail
à partir
du
janvier
2018
La présente convention collective de travail entre en vigueur le 1er janvier 2018
La présente
convention collective de travail
entre en vigueur
le
janvier
2018
La présente convention collective de travail entre en vigueur le 1er janvier 2018 pour les allocations afférentes à l'exercice 2018 et cesse d'être en vigueur le 31 décembre 2018
La présente
convention collective de travail
cesse d'être en vigueur
le
décembre
2018
La présente convention collective de travail produit ses effets le 1” juillet 2018
La présente
convention collective de travail
produit ses effets
le
juillet
2018
La présente convention collective de travail entre en vigueur le 1” juillet 2018
La présente
convention collective de travail
entre en vigueur
le
juillet
2018


# Preprocessing for NLP

In [ ]:
from pandas import Series

from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer

# import entirely spacy to create Doc objects through nlp
import spacy
from spacy import load, lang

from wordcloud import WordCloud

from collections import Counter

from typing import List
from typing import Dict

# WARNINGS
# W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
# I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine

# GLOBAL VARIABLES
NAMED_ENTITIES = ['microsoft']

#testing
from os import getcwd as cwd
from os.path import dirname as dir
from os.path import join
import pandas as pd

In [ ]:
def lemmatize(text_tokens: List[str]) -> List[str]:
    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)

    # Instantiate the WordNetLemmatizer
    wordnet_lemmatizer = WordNetLemmatizer()
    # Lemmatize all tokens into a new list: lemmatized
    texts_lemmatized = [wordnet_lemmatizer.lemmatize(t, get_wordnet_pos(t)) for t in text_tokens]
    return texts_lemmatized


def filter_words(texts_lemmatized: List[List[str]], freq_min=None, freq_max=None):
    frequency_absolute = Counter([item for elem in texts_lemmatized for item in elem])
    wordcloud = WordCloud(width=1000, height=500).generate_from_frequencies(frequency_absolute)
    frequency_relative = wordcloud.words_
    if freq_min is not None and freq_min > 0 and freq_min < 1:
        rel_freq_filtered = {k: v for k, v in frequency_relative.items() if v > freq_min}
    if freq_max is not None and freq_max > 0 and freq_max < 1:
        rel_freq_filtered = {k: v for k, v in frequency_relative.items() if v < freq_max}
    texts_filtered = [[t for t in pub_lem if t in rel_freq_filtered.keys()] for pub_lem in texts_lemmatized]
    # testing part
    f_abs_updated =Counter([ item for elem in texts_filtered for item in elem])   
    wordcloud_updated = WordCloud(width=1000, height=500).generate_from_frequencies(f_abs_updated)
    f_rel_updated = wordcloud_updated.words_
    return texts_filtered, f_abs_updated, f_rel_updated, wordcloud_updated


In [ ]:
class Preprocess:
    def __init__(self, nlp_model='en_core_web_md'):
        self.nlp = load(nlp_model)
        if 'en_' in nlp_model:
          self.stop_words = lang.en.stop_words.STOP_WORDS
        elif 'fr_' in nlp_model:
          self.stop_words = lang.fr.stop_words.STOP_WORDS
        self.named_entities = set(NAMED_ENTITIES)

    def get_named_entities(self, texts: Series, inplace=True) -> set:
        # creating a single ner set
        nes = set()
        # function to extract NER from text
        def get_named_entities(text) -> set:
            doc = self.nlp(text)
            named_entities = set([ent.text for ent in doc.ents])
            return named_entities
        [[nes.add(n) for n in get_named_entities(text)] for text in texts]
        # adding predefined NER
        [nes.add(n) for n in self.named_entities]
        if inplace:
            self.named_entities = nes
        return nes

    def tokenize_text(self, text:str, stop_words: List[str] = None, named_entities: List[str] = None,
                   lenght_min: int=2) -> List[str]:
        if stop_words is None:
            stop_words = self.stop_words
        if named_entities is None:
            named_entities = self.named_entities
        text = text.replace("\n", " ")
        # split string into words (tokens)
        tokens = word_tokenize(text.lower())
        # keep strings with only alphabets
        tokens = [t for t in tokens if t.isalpha()]
        tokens = lemmatize(tokens)
        # remove short words, they're probably not useful
        tokens = [t for t in tokens if len(t) > lenght_min]
        # remove stopwords
        tokens = [t for t in tokens if t not in stop_words]
        # remove
        tokens = [t for t in tokens if t not in named_entities]
        return tokens

    def clean_text(self, text:str, stop_words: List[str] = None, named_entities: List[str] = None,
                   lenght_min: int=2) -> str:
        tokens = self.tokenize_text(text, stop_words, named_entities, lenght_min)
        text_cleaned = " ".join(tokens)
        return text_cleaned

    def tokenize_texts(self, texts:Series, stop_words: List[str] = None, named_entities: List[str] = None,
                   lenght_min: int=2) -> List[List[str]]:
        texts_tokens = []
        for text in texts:
            texts_tokens += [self.tokenize_text(text, stop_words, named_entities, lenght_min)]
        return texts_tokens

In [ ]:
! python -m spacy download fr_core_news_md

In [ ]:
import spacy
nlp = spacy.load('fr_core_news_md')

In [ ]:
preprocess = Preprocess(nlp_model='fr_core_news_md')

In [ ]:
# example of FR stopwords from the set
list(preprocess.stop_words)[:10]

In [ ]:
# importing nltk and downloading additional required packages
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
#cleaning text before extracting word clouds
responses2["bodies_cleaned"] = responses2["doc_bodies"].apply(preprocess.clean_text)

# Word cloud labelling

In [ ]:
#extracting all words with relative frequency > 0.5
all_ts = preprocess.tokenize_texts(responses2.bodies_cleaned)

In [ ]:
texts, f_abs, f_rel, wordcloud = filter_words(all_ts, freq_min=0.5)

In [ ]:
wordcloud.to_image()

In [ ]:
# adding most frequent words as stop words
[preprocess.stop_words.add(w) for w in list(f_rel.keys())]
[w for w in list(preprocess.stop_words) if w in list(f_rel.keys())]

In [ ]:
[w for w in list(preprocess.stop_words) if w in list(f_rel.keys())]

In [ ]:
themes_keywords = pd.DataFrame(columns=['themeFr', 'frequency_absolute', 'frequency_relative'])
for t in themes_fr:
  mask = responses2.dropna(axis=0, subset=['themesFr']).themesFr.map(lambda x: t in x)
  r_t = responses2.dropna(axis=0, subset=['themesFr'])[mask]
  t_ts = preprocess.tokenize_texts(r_t["bodies_cleaned"])
  t_texts, t_f_abs, t_f_rel, t_wordcloud = filter_words(t_ts, freq_min=0.05, freq_max=0.95)
  themes_keywords = themes_keywords.append({'themeFr':t, 'frequency_absolute':t_f_abs, 'frequency_relative':t_f_rel}, ignore_index=True)
  print(f'Kewyords for {t} added, {len(r_t)} documents found')

In [ ]:
for t in themes_fr:
  mask = responses2.dropna(axis=0, subset=['themesFr']).themesFr.map(lambda x: t in x)
  r_t = responses2.dropna(axis=0, subset=['themesFr'])[mask]
  print(f'{len(r_t)} contained for {t}')

In [ ]:
themes_keywords.to_csv('/content/drive/My Drive/Becode/Kpmg/labeling.csv')

In [ ]:
themes_keywords.loc[:, ['themeFr','frequency_relative']]

# Classification model

In [ ]:
from pickle import load

with open(os.path.join(KPMG_PATH, "clas_new.pkl"), 'rb') as f:
  df = load(f)

In [ ]:
df.columns

In [ ]:
df["body_cleaned"] = df["docBodyFr"].apply(preprocess.clean_text)

In [ ]:
#generating all possible FR themes (replacing previous list of 53 items)
themes_fr = []
for r in  df.themesFr:
    if r is not None:
        for t in r:
            if t not in themes_fr:
                themes_fr += [t]
len(themes_fr)

In [ ]:
# showing only 27 unique themes against 54 available
df.dropna(axis=0, subset=['themesFr'], inplace=True)
df['theme'] = df['themesFr'].apply(lambda x: x[0] if len(x) == 1 else None)
len(df.theme.unique())

In [ ]:
for t in themes_fr:
	df.dropna(axis=0, subset=['themesFr'], inplace=True)
	df[t] = df['themesFr'].apply(lambda x: 1 if t in x else 0)
df.columns